# Using FISSA with SIMA

[SIMA](http://www.losonczylab.org/sima/) is a toolbox for motion correction and cell detection.
Here we illustrate how to create a workflow which uses SIMA to detect cells and FISSA to extract decontaminated signals from those cells.

**Reference:**
Kaifosh, P., Zaremba, J. D., Danielson, N. B., Losonczy, A., 2014. SIMA: Python software for analysis of dynamic fluorescence imaging data. Frontiers in neuroinformatics 8 (80). doi:&nbsp;[10.3389/fninf.2014.00080](https://doi.org/10.3389/fninf.2014.00080)

## Imports

In [ ]:
# FISSA toolbox
import fissa

# SIMA toolbox
import sima
import sima.segment

# File operations
import glob

# Plotting toolbox, with notebook embedding options
import holoviews as hv

%load_ext holoviews.ipython
%output widgets="embed"

## Detecting cells with SIMA

### Setup data

In [ ]:
# Define folder where tiffs are present
tiff_folder = "exampleData/20150529/"

# Find tiffs in folder
tiffs = sorted(glob.glob(tiff_folder + "/*.tif*"))

# define motion correction method
mc_approach = sima.motion.DiscreteFourier2D()

# Define SIMA dataset
sequences = [sima.Sequence.create("TIFF", tiff) for tiff in tiffs[:1]]
try:
    dataset = sima.ImagingDataset(sequences, "example.sima")
except:
    dataset = sima.ImagingDataset.load("example.sima")

### Run SIMA segmentation algorithm

In [ ]:
stica_approach = sima.segment.STICA(components=2)
stica_approach.append(sima.segment.SparseROIsFromMasks())
stica_approach.append(sima.segment.SmoothROIBoundaries())
stica_approach.append(sima.segment.MergeOverlapping(threshold=0.5))
rois = dataset.segment(stica_approach, "auto_ROIs")

### Plot detected cells

In [ ]:
fig = hv.Overlay()
for roi in rois:
    fig *= hv.Curve(roi.coords[0])

fig

## Extract decontaminated signals with FISSA

FISSA needs either ImageJ ROIs or numpy arrays as inputs for the ROIs. 

SIMA outputs ROIs as numpy arrays, and can be directly read into FISSA.

A given roi is given as
```python
rois[i].coords[0][:, :2]
```

FISSA expects rois to be provided as a list of lists
```
[[roiA1, roiA2, roiA3, ...]]
```
So some formatting will need to be done first.

In [ ]:
numROI = len(rois)
rois_fissa = [roi.coords[0][:, :2] for roi in rois]

In [ ]:
rois[0].coords[0][:, :2].shape

We can then run FISSA on the data using the ROIs supplied by SIMA having converted them to a FISSA-compatible format, `rois_fissa`.

In [ ]:
output_folder = "fissa_sima_example"
exp = fissa.Experiment(tiff_folder, [rois_fissa], output_folder)
exp.separate()

### Plotting the results

In [ ]:
%%opts Curve {+axiswise}


def plot_cell_regions(roi_polys, plot_neuropil=False):
    """
    Plot a single cell region, using holoviews.
    """
    out = hv.Overlay()

    if plot_neuropil:
        # Plot the neuropil as well as the ROI
        n_region = len(roi_polys)
    else:
        # Just plot the ROI, not the neuropil
        n_region = 1

    for i_region in range(n_region):
        x = roi_polys[i_region][0][:, 1]
        y = roi_polys[i_region][0][:, 0]
        out *= hv.Curve(zip(x, y))

    return out


i_trial = 0

# Generate outlines around for all detected regions, indicating neuropil subregions
region_plots = {
    i_cell: plot_cell_regions(exp.roi_polys[i_cell, i_trial], plot_neuropil=True)
    for i_cell in range(exp.nCell)
}

# Generate plots for raw extracts and neuropil removed
traces_plots = {
    i_cell: (
        hv.Curve(exp.raw[i_cell][i_trial][0, :], label="SIMA")
        * hv.Curve(exp.result[i_cell][i_trial][0, :], label="FISSA")
    )
    for i_cell in range(exp.nCell)
}

# Get average image
avg_img = hv.Raster(exp.means[i_trial])

# Render holoviews
(
    avg_img * hv.HoloMap(region_plots, kdims=["Cell"]) * fig
    + hv.HoloMap(traces_plots, kdims=["Cell"])
)

(A) ROI contours, and the neuropil subregions defined by FISSA for the current cell.

(B) Signal extracted by SIMA's detected ROI (blue), and after decontaminating with FISSA (red)